In [2]:
\l ../querqus.q

[0;31m../querqus.q. OS reports: No such file or directory[0m: [0;31m../querqus.q. OS reports: No such file or directory[0m

In [1]:
/ quercus: parser combinators for q
\d .qu

ret:{enlist(x;y)};
bind:{raze({(a;s):y;x[a]s}[x]')y@};
map:{bind[(ret x::)]};
trav:{({bind[{map[{raze(x;y)}[y]][x]}[y]]x}/)(x')y};
seqA:trav[::];
zero:{[x]()};
plus:{x[z],y z};
fil:{bind[{(zero;ret y)x y}[x]][y]};
opt:{plus[x;ret()]};
many:{plus[bind[{map[(enlist[z],)]y x}[x;.z.s];x];ret()]};
many1:{bind[{map[(enlist[y],)]many x}[x];x]};
times:{$[x<1;ret();seqA x#y]};
sep1:{bind[{map[{enlist[x],y}[z]]many seqr[x;y]}[y;x]]x};
sep:{plus[sep1[x;y];ret()]};
skip:map zero;
item:{$[""~x;();enlist(first x;1_ x)]};
seqf:{bind[{map[{x(y;z)}[x;z]][y]}[x;z]]y};
seql:seqf[first];seqr:seqf[last];seq:seqf[enlist .];
sat:{bind[{$[x y;ret y;zero]}[x];item]};
oneof:{sat in[;x]};
noneof:{sat(not in[;x]::)};
between:{seqr[x;seql[z;y]]};
range:{sat{(x<=z)&z<=y}[x;y]};
digit:range ."09";
lwr:range ."az";
upr:range ."AZ";
letter:plus[lwr;upr];
alphanum:plus[letter;digit];
str:{$[x~count[x]#y;enlist(x;count[x]_y);()]};
word:many1 letter;
num:many1 digit;
chr:{[x:`c]sat[(x=)]};
spaces:skip many space:chr" ";
eof:{$[""~x;ret[()]x;zero x]};
parens:between[chr"(";chr")"];
braces:between[chr"{";chr"}"];
c:item;
j:map[("J"$)]num;
s:map[(`$)]word;

rparse:{$[()~r:x y;'`parse;1<count r;'`ambig;[(a;s):r 0;not ""~s];'`spare;a]};
vparse:{.[{[x]1b}rparse::;(x;y);0b]};

\d .

In [ ]:
// parse: encontró lo que no debía
// spare: quedan cosas por parsear
// ambig: ambiguedad

In [2]:
test:{z~.[.qu.rparse;(x;y);::]}

In [3]:
test[.qu.lwr;enlist"a";"a"]

1b


In [4]:
test[.qu.lwr;"ab";"spare"]

1b


In [5]:
testtime:{[parser;input;expected;times]
    do[times;r,:(system"t [parser][input]")];
    expected>=avg r}

In [5]:
testtime[.qu.times[2;.qu.lwr];"ab";1;30]

[0;31minput[0m: [0;31minput[0m

In [34]:
/ si se guardan parser e input como globales la celda anterior funciona
/ quizá los "fstring" tiran de variables globales. si es el caso no interesa
/ habría que buscar una alternativa... (o no)
parser:.qu.lwr
input:enlist"a"

system"t [parser][input]"

0


In [24]:
flip (!). flip((`foo;(3;3;4));
     (`bar;(4;4;4));
     (`baz;(5;5;4)))

foo bar baz
-----------
3   4   5  
3   4   5  
4   4   4  


In [19]:
timest:flip(!). flip((`func;`times);
                    (`parser;(".qu.times[2;.qu.digit]";".qu.times[3;.qu.digit]"));
                    (`inputs;("12";"123"));
                    (`expected;("12";"spare")))

In [20]:
timest

func  parser                   inputs expected
----------------------------------------------
times ".qu.times[2;.qu.digit]" "12"   "12"    
times ".qu.times[3;.qu.digit]" "123"  "spare" 


In [22]:
{expected~(eval parse parser)[input]}

"12" ""
